<a href="https://colab.research.google.com/github/JayeonKangNature/Bachelor_Project_Thesis_2023/blob/main/Error_Counting_for_Medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For mouting
from google.colab import drive
import torch
import os
import json

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import torch
import os
# Base path

Annots_Path = '/content/drive/MyDrive/BSc_Project/Main_code_folder/TIMOTHY/val_300.json'
import json
import random

# Load the COCO JSON file
with open(Annots_Path, 'r') as f:
    coco_data = json.load(f)

import json
import random

# Load the COCO JSON file
with open(Annots_Path, 'r') as f:
    coco_data = json.load(f)
Collect_image_ids = list(coco_data.keys())
Collect_image_ann = list(coco_data.values())

Test_ = dict(zip(Collect_image_ids, Collect_image_ann))
with open('test.json', 'w') as f:
    json.dump(Test_, f)

with open('test.json', 'r') as test:
    Test_coco_data = json.load(test)

In [ ]:
import json
from pathlib import Path
from typing import Tuple

import pandas as pd

DATA_PATH = Path()
images_path = Path("/content/drive/MyDrive/BSc_Project/Val_300/images")

def load_dataset(
    data_path: Path = DATA_PATH,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Read the COCO style json dataset and transform it into convenient DataFrames
    :return (images_df, targets_df):
        images_df: Columns "image_id" and "file_name"
        targets_df: Columns
            "target_id", "image_id", "xmin", "ymin", "xmax", "ymax", "label_id"
    """
    annotations_path = '/content/drive/MyDrive/BSc_Project/Main_code_folder/TIMOTHY/val_300.json'

    with open(annotations_path, "r") as f:
        targets_json = json.load(f)

    images_df = pd.DataFrame.from_records(targets_json["images"])
    images_df.rename(columns={"id": "image_id"}, inplace=True)
    images_df = images_df[["image_id", "file_name"]]

    targets_df = pd.DataFrame.from_records(targets_json["annotations"])
    targets_df[["xmin", "ymin", "w", "h"]] = targets_df["bbox"].tolist()
    targets_df["xmax"] = targets_df["xmin"] + targets_df["w"]
    targets_df["ymax"] = targets_df["ymin"] + targets_df["h"]
    targets_df.reset_index(inplace=True)
    targets_df.rename(
        columns={"index": "target_id", "category_id": "label_id"}, inplace=True
    )
    targets_df = targets_df[
        ["target_id", "image_id", "label_id", "xmin", "ymin", "xmax", "ymax"]
    ]

    return images_df, targets_df

images_df, targets_df = load_dataset(DATA_PATH)

In [ ]:
print(images_df)

     image_id                      file_name
0           1  cju7bb3ss1uoo0755pmhyco7t.jpg
1           2  cju7bc95p1mdm0817yqj5jc6j.jpg
2           3  cju7bd1qu1mx409877xjxibox.jpg
3           4  cju7bduyq1rjf08719giru9ho.jpg
4           5  cju7bf1lp1shi081835vs84lc.jpg
..        ...                            ...
295       296  cju8b2rmgr52s0801p54eyflx.jpg
296       297  cju8bj2ssrmlm0871gc2ug2rs.jpg
297       298  cju8c1a0ws7o208181c6lbsom.jpg
298       299  cju8cgi2kspp308011nxdtjp6.jpg
299       300  cju8doa16u5gh0818w1ywda3q.jpg

[300 rows x 2 columns]


In [ ]:
print(targets_df)

     target_id  image_id  label_id  xmin  ymin  xmax  ymax
0            0         1         1    79   128   341   354
1            1         2         1   236   198   420   425
2            2         3         1   158   157   355   352
3            3         4         1   247   221   466   444
4            4         5         1    24   108   424   530
..         ...       ...       ...   ...   ...   ...   ...
310        310       296         1   172   114   476   364
311        311       297         1   274   184   311   222
312        312       298         1   138     0   624   341
313        313       299         1   373   242   487   357
314        314       300         1     0     0   621   528

[315 rows x 7 columns]


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/BSc_Project/Main_code_folder/TIMOTHY/preds_df_faster_medical.csv")
preds_df = data


In [ ]:
print(preds_df)

     Unnamed: 0  pred_id  image_id  label_id        xmin        ymin  \
0             0        0         1         1   81.019051  118.619362   
1             1        1         2         1  211.590256  128.473343   
2             2        2         2         1  176.756439  141.900314   
3             3        3         2         1  436.286530  109.935310   
4             4        4         3         1  155.735779  149.703842   
..          ...      ...       ...       ...         ...         ...   
714         714      714       299         1  361.220520  241.604340   
715         715      715       300         1  492.387543  194.344894   
716         716      716       300         1   42.623508  130.484741   
717         717      717       300         1  204.823456   37.226707   
718         718      718       300         1  357.373169  146.806686   

           xmax        ymax     score  
0    339.983803  336.566750  0.951372  
1    445.292389  429.909988  0.916277  
2    383.872955

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/BSc_Project/Main_code_folder/TIMOTHY/preds_df_yolo_medical.csv")
preds_df_yolo = data

In [ ]:
# Code from gist: https://gist.github.com/bepuca/8014e9b40e92ecc99574d9510a8608f2
from typing import Dict, Set, Tuple

import numpy as np
import pandas as pd
import torch
import torchvision

TARGETS_DF_COLUMNS = [
    "target_id",
    "image_id",
    "label_id",
    "xmin",
    "ymin",
    "xmax",
    "ymax",
]
PREDS_DF_COLUMNS = [
    "pred_id",
    "image_id",
    "label_id",
    "xmin",
    "ymin",
    "xmax",
    "ymax",
    "score",
]
ERRORS_DF_COLUMNS = ["pred_id", "target_id", "error_type"]

BACKGROUND_IOU_THRESHOLD = 0.1
FOREGROUND_IOU_THRESHOLD = 0.5


class ErrorType:
    OK = "correct"  # pred -> IoU > foreground; target_label == pred_label; highest score
    CLS = "classification"  # pred -> IoU > foreground; target_label != pred_label
    LOC = "localization"  # pred -> background < IoU < foreground; target_label == pred_label
    CLS_LOC = "cls & loc"  # pred -> background < IoU < foreground; target_label != pred_label
    DUP = "duplicate"  # pred -> background < IoU < foreground; target_label != pred_label
    BKG = "background"  # pred -> IoU > foreground; target_label == pred_label; no highest score
    MISS = "missed"  # target -> No pred with Iou > background


def classify_predictions_errors(
    targets_df: pd.DataFrame,
    preds_df: pd.DataFrame,
    iou_background: float = BACKGROUND_IOU_THRESHOLD,
    iou_foreground: float = FOREGROUND_IOU_THRESHOLD,
) -> pd.DataFrame:
    """Classify predictions

    We assume model is right as much as possible. Thus, in case of doubt
    (i.e matching two targets), a prediction will be first considered
    ErrorType.LOC before ErrorType.CLS.

    The error definition credit belongs to the following paper (refer to it for
    conceptual details):
        TIDE: A General Toolbox for Identifying Object Detection Errors
        https://arxiv.org/abs/2008.08115

    :param targets_df: DataFrame with all targets for all images with TARGETS_DF_COLUMNS.
    :param preds_df: DataFrame with all predictions for all images with PREDS_DF_COLUMNS.
    :param iou_background: Minimum IoU for a prediction not to be considered background.
    :param iou_foreground: Minimum IoU for a prediction to be considered foreground.
    :return errors_df: DataFrame with all error information with ERRORS_DF_COLUMNS
    """

    # Provide clarity on expectations and avoid confusing errors down the line
    assert (set(TARGETS_DF_COLUMNS) - set(targets_df.columns)) == set()
    assert (set(PREDS_DF_COLUMNS) - set(preds_df.columns)) == set()

    pred2error = dict()  # {pred_id: ErrorType}
    target2pred = (
        dict()
    )  # {target_id: pred_id}, require iou > iou_foreground & max score
    pred2target = dict()  # {pred_id: target_id}, require iou >= iou_background
    missed_targets = set()  # {target_id}

    # Higher scoring preds take precedence when multiple fulfill criteria
    preds_df = preds_df.sort_values(by="score", ascending=False)

    for image_id, im_preds_df in preds_df.groupby("image_id"):
        # Need to reset index to access dfs with same idx we access
        #   IoU matrix down the line
        im_targets_df = targets_df.query("image_id == @image_id").reset_index(
            drop=True
        )
        im_preds_df = im_preds_df.reset_index(drop=True)

        if im_targets_df.empty:
            pred2error = {**pred2error, **_process_empty_image(im_preds_df)}
        else:
            iou_matrix, iou_label_match_matrix = _compute_iou_matrices(
                im_targets_df, im_preds_df
            )

            # Iterate over all predictions. Higher scores first
            for pred_idx in range(len(im_preds_df)):
                match_found = _match_pred_to_target_with_same_label(
                    pred_idx,
                    pred2error,
                    pred2target,
                    target2pred,
                    iou_label_match_matrix,
                    im_targets_df,
                    im_preds_df,
                    iou_background,
                    iou_foreground,
                )
                if match_found:
                    continue

                _match_pred_wrong_label_or_background(
                    pred_idx,
                    pred2error,
                    pred2target,
                    iou_matrix,
                    im_targets_df,
                    im_preds_df,
                    iou_background,
                    iou_foreground,
                )

    missed_targets = _find_missed_targets(targets_df, pred2target)
    errors_df = _format_errors_as_dataframe(
        pred2error, pred2target, missed_targets
    )
    return errors_df[list(ERRORS_DF_COLUMNS)]


def _process_empty_image(im_preds_df: pd.DataFrame) -> Dict[int, str]:
    """In an image without targets, all predictions represent a background error"""
    return {
        pred_id: ErrorType.BKG for pred_id in im_preds_df["pred_id"].unique()
    }


def _compute_iou_matrices(
    im_targets_df: pd.DataFrame, im_preds_df: pd.DataFrame
) -> Tuple[np.array, np.array]:
    """Compute IoU matrix between all targets and preds in the image

    :param im_targets_df: DataFrame with targets for the image being processed.
    :param im_preds_df: DataFrame with preds for the image being processed.
    :return:
        iou_matrix: Matrix of size (n_targets, n_preds) with IoU between all
            targets & preds
        iou_label_match_matrix: Same as `iou_matrix` but 0 for all target-pred
            pair with different labels (i.e. IoU kept only if labels match).
    """
    # row indexes point to targets, column indexes to predictions
    iou_matrix = iou_matrix = torchvision.ops.box_iou(
        torch.from_numpy(
            im_targets_df[["xmin", "ymin", "xmax", "ymax"]].values
        ),
        torch.from_numpy(im_preds_df[["xmin", "ymin", "xmax", "ymax"]].values),
    ).numpy()

    # boolean matrix with True iff target and pred have the same label
    label_match_matrix = (
        im_targets_df["label_id"].values[:, None]
        == im_preds_df["label_id"].values[None, :]
    )
    # IoU matrix with 0 in all target-pred pairs that have different label
    iou_label_match_matrix = iou_matrix * label_match_matrix
    return iou_matrix, iou_label_match_matrix


def _match_pred_to_target_with_same_label(
    pred_idx: int,
    pred2error: Dict[int, str],
    pred2target: Dict[int, int],
    target2pred: Dict[int, int],
    iou_label_match_matrix: np.array,
    im_targets_df: pd.DataFrame,
    im_preds_df: pd.DataFrame,
    iou_background: float,
    iou_foreground: float,
) -> bool:
    """Try to match `pred_idx` to a target with the same label and identify error (if any)

    If there is a match `pred2error`, `pred2target` and (maybe) `target2pred`
    are modified in place.
    Possible error types found in this function:
        ErrorType.OK, ErrorType.DUP, ErrorType.LOC

    :param pred_idx: Index of prediction based on score (index 0 is maximum score for image).
    :param pred2error: Dict mapping pred_id to error type.
    :param pred2target: Dict mapping pred_id to target_id (if match found with iou above background)
    :param target2pred: Dict mapping target_id to pred_id to pred considered correct (if any).
    :param iou_label_match_matrix: Matrix with size [n_targets, n_preds] with IoU between all preds
        and targets that share label (i.e. IoU = 0 if there is a label missmatch).
    :param im_targets_df: DataFrame with targets for the image being processed.
    :param im_preds_df: DataFrame with preds for the image being processed.
    :param iou_background: Minimum IoU to consider a pred not background for target.
    :param iou_foreground: Minimum IoU to consider a pred foreground for a target.
    :return matched: Whether or not there was a match and we could identify the pred error.
    """
    # Find highest overlapping target for pred processed
    target_idx = np.argmax(iou_label_match_matrix[:, pred_idx])
    iou = np.max(iou_label_match_matrix[:, pred_idx])
    target_id = im_targets_df.at[target_idx, "target_id"]
    pred_id = im_preds_df.at[pred_idx, "pred_id"]

    matched = False
    if iou >= iou_foreground:
        pred2target[pred_id] = target_id
        # Check if another prediction is already the match for target to
        #   identify duplicates
        if target2pred.get(target_id) is None:
            target2pred[target_id] = pred_id
            pred2error[pred_id] = ErrorType.OK
        else:
            pred2error[pred_id] = ErrorType.DUP
        matched = True

    elif iou_background <= iou < iou_foreground:
        pred2target[pred_id] = target_id
        pred2error[pred_id] = ErrorType.LOC
        matched = True
    return matched


def _match_pred_wrong_label_or_background(
    pred_idx: int,
    pred2error: Dict[int, str],
    pred2target: Dict[int, int],
    iou_matrix: np.array,
    im_targets_df: pd.DataFrame,
    im_preds_df: pd.DataFrame,
    iou_background: float,
    iou_foreground: float,
) -> None:
    """Try to match `pred_idx` to a target (with different label) and identify error

    If there is a match `pred2error` and  (maybe) `pred2target` are modified in place.
    Possible error types found in this function:
        ErrorType.BKG, ErrorType.CLS, ErrorType.CLS_LOC

    :param pred_idx: Index of prediction based on score (index 0 is maximum score for image).
    :param pred2error: Dict mapping pred_id to error type.
    :param pred2target: Dict mapping pred_id to target_id (if match found with iou above background)
    :param target2pred: Dict mapping target_id to pred_id to pred considered correct (if any).
    :param iou: Matrix with size [n_targets, n_preds] with IoU between all preds and targets.
    :param im_targets_df: DataFrame with targets for the image being processed.
    :param im_preds_df: DataFrame with preds for the image being processed.
    :param iou_background: Minimum IoU to consider a pred not background for target.
    :param iou_foreground: Minimum IoU to consider a pred foreground for a target.
    """
    # Find highest overlapping target for pred processed
    target_idx = np.argmax(iou_matrix[:, pred_idx])
    iou = np.max(iou_matrix[:, pred_idx])
    target_id = im_targets_df.at[target_idx, "target_id"]
    pred_id = im_preds_df.at[pred_idx, "pred_id"]

    if iou < iou_background:
        pred2error[pred_id] = ErrorType.BKG

    # preds with correct label do not get here. Thus, no need to check if label
    #   is wrong
    elif iou >= iou_foreground:
        pred2target[pred_id] = target_id
        pred2error[pred_id] = ErrorType.CLS
    else:
        # No match to target, as we cannot be sure model was remotely close to
        #   getting it right
        pred2error[pred_id] = ErrorType.CLS_LOC


def _find_missed_targets(
    im_targets_df: pd.DataFrame, pred2target: Dict[int, int]
) -> Set[int]:
    """Find targets in the processed image that were not matched by any prediction

    :param im_targets_df: DataFrame with targets for the image being processed.
    :param pred2target: Dict mapping pred_id to target_id (if match found with
        iou above background)
    :return missed_targets: Set of all the target ids that were missed

    """
    matched_targets = [t for t in pred2target.values() if t is not None]
    missed_targets = set(im_targets_df["target_id"]) - set(matched_targets)
    return missed_targets


def _format_errors_as_dataframe(
    pred2error: Dict[int, str],
    pred2target: Dict[int, int],
    missed_targets: Set[int],
) -> pd.DataFrame:
    """Use the variables used to classify errors to format them in a ready to use DataFrame

    :param pred2error: Dict mapping pred_id to error type.
    :param pred2target: Dict mapping pred_id to target_id (if match found with
        iou above background)
    :param missed_targets: Set of all the target ids that were missed
    :return: DataFrame with columns ERRORS_DF_COLUMNS
    """
    errors_df = pd.DataFrame.from_records(
        [
            {"pred_id": pred_id, "error_type": error}
            for pred_id, error in pred2error.items()
        ]
    )
    errors_df["target_id"] = None
    errors_df.set_index("pred_id", inplace=True)
    for pred_id, target_id in pred2target.items():
        errors_df.at[pred_id, "target_id"] = target_id

    missed_df = pd.DataFrame(
        {
            "pred_id": None,
            "error_type": ErrorType.MISS,
            "target_id": list(missed_targets),
        }
    )
    errors_df = pd.concat(
        [errors_df.reset_index(), missed_df], ignore_index=True
    ).astype(
        {"pred_id": float, "target_id": float, "error_type": pd.StringDtype()}
    )
    return errors_df

In [ ]:
errors_df = classify_predictions_errors(targets_df, preds_df)

In [ ]:
errors_df_yolo = classify_predictions_errors(targets_df, preds_df_yolo)

In [ ]:
print(errors_df)

     pred_id  target_id    error_type
0        0.0        0.0       correct
1        1.0        1.0       correct
2        2.0        1.0  localization
3        3.0        NaN    background
4        4.0        2.0       correct
..       ...        ...           ...
721      NaN      261.0        missed
722      NaN       16.0        missed
723      NaN      276.0        missed
724      NaN      184.0        missed
725      NaN       29.0        missed

[726 rows x 3 columns]


In [ ]:
print(errors_df_yolo)

      pred_id  target_id  error_type
0      2166.0        0.0     correct
1      2167.0        NaN  background
2      2168.0        NaN  background
3      2169.0        NaN  background
4      2170.0        NaN  background
...       ...        ...         ...
2194      NaN      197.0      missed
2195      NaN      294.0      missed
2196      NaN      276.0      missed
2197      NaN      154.0      missed
2198      NaN       29.0      missed

[2199 rows x 3 columns]


In [ ]:
errors_df["error_type"].value_counts()

correct         279
localization    221
background      219
missed            7
Name: error_type, dtype: Int64

In [ ]:
errors_df_yolo["error_type"].value_counts()

background      1382
localization     503
correct          273
duplicate         35
missed             6
Name: error_type, dtype: Int64

In [ ]:
import pandas as pd

# Assuming you have a DataFrame called preds_df

# Specify the file path and name for the CSV file
file_path = "/content/drive/MyDrive/BSc_Project/Main_code_folder/TIMOTHY/errors_df_faster_medical.csv"

# Save the DataFrame as a CSV file
errors_df.to_csv(file_path, index=False)

In [ ]:
import pandas as pd

# Assuming you have a DataFrame called preds_df

# Specify the file path and name for the CSV file
file_path = "/content/drive/MyDrive/BSc_Project/Main_code_folder/TIMOTHY/errors_df_yolo_medical.csv"

# Save the DataFrame as a CSV file
errors_df_yolo.to_csv(file_path, index=False)